In [12]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd
import numpy as np
from snowflake.snowpark.functions import sproc

# lets import some tranformations functions
from snowflake.snowpark.functions import udf, col, lit, translate, is_null, iff

In [10]:
from config import snowflake_procedure
from snowflake.snowpark import version
print(version.VERSION)
#session.close()
session = Session.builder.configs(snowflake_procedure).create()
session.sql("use role accountadmin").collect()
session.sql("create database if not exists  {}".format(snowflake_procedure['database'])).collect()
session.sql("use database {}".format(snowflake_procedure['database'])).collect()
session.sql("create schema if not exists {}".format(snowflake_procedure['schema'])).collect()
session.sql("use schema {}".format(snowflake_procedure['schema'])).collect()
session.sql("create or replace warehouse {} with \
                WAREHOUSE_SIZE = XSMALL \
                AUTO_SUSPEND = 120 \
                AUTO_RESUME = TRUE".format(snowflake_procedure['warehouse'])).collect()
session.sql("use warehouse {}".format(snowflake_procedure['warehouse']))
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(0, 8, 0)
[Row(CURRENT_WAREHOUSE()='SP_QS_WH', CURRENT_DATABASE()='PERSONA', CURRENT_SCHEMA()='KAFKA_SCHEMA')]


In [3]:
df_kafka = session.table('SQLSAWIN_DBO_PERSONA_1663494945')

In [5]:
df_kafka.show()

-----------------------------------------------------------------------------------------------------------
|"RECORD_METADATA"                                   |"RECORD_CONTENT"                                    |
-----------------------------------------------------------------------------------------------------------
|{                                                   |{                                                   |
|  "CreateTime": 1665681402034,                      |  "payload": {                                      |
|  "key": "{\"schema\":{\"type\":\"struct\",\"fi...  |    "after": {                                      |
|  "offset": 6,                                      |      "id": 8,                                      |
|  "partition": 0,                                   |      "lastname": "borges    ",                     |
|  "topic": "sqlsawin.dbo.PERSONA"                   |      "name": "andre     "                          |
|}                          

In [6]:
df_kafka.queries

{'queries': ['SELECT  *  FROM (SQLSAWIN_DBO_PERSONA_1663494945)'],
 'post_actions': []}

In [13]:
df_message = session.table('SQLSAWIN_DBO_PERSONA_1663494945').select(col("RECORD_CONTENT"))

In [23]:
print(df_message.count())

12


In [26]:
df_pandas = df_message.to_pandas()

In [31]:
dict(df_pandas.iloc[[2]])

{'RECORD_CONTENT': 2    {\n  "payload": {\n    "after": {\n      "id":...
 Name: RECORD_CONTENT, dtype: object}

In [32]:
session.close()